##### This is my training project to learn LangGraph agents, the idea is to create an Agent with different tools, then ask it different things and check if is using the tools and the process.


In [ ]:
# Load variables
from dotenv import load_dotenv 
load_dotenv()
import getpass
import os

if not os.environ.get("GOOGLE_API_KEY"):
  os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass()

In [ ]:
# Define the model
from langchain.chat_models import init_chat_model

model = init_chat_model("gemini-2.0-flash", model_provider="google_genai")
